In [1]:
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew , variation
from scipy.signal import welch
import pandas as pd
from scipy.signal import hilbert
import glob
from mne.event import define_target_events

In [2]:
#подгружаем библиотитеку которая октрывает ЭЭГ(картинки) в отдельном окне 
%matplotlib qt

In [3]:
import os

In [4]:
os.chdir('D:/all_eeg/mu_eeg/chernovik_2/ssd/')

In [5]:
filenames = glob.glob('*')#кавычокой я опредляю тип файлов для просмотра 
filenames

['NP102302',
 'NP102303',
 'NP102304',
 'NP102305',
 'NP102306',
 'NP102307',
 'NP102309',
 'NP102310',
 'NP102311',
 'NP102312',
 'NP102313',
 'NP102314',
 'NP102315',
 'NP112301',
 'NP112302',
 'NP112303',
 'NP112304',
 'NP112306',
 'NP112307',
 'NP112308',
 'NP112309',
 'NP112310',
 'NP112311',
 '~$мментарии (автовосстановление).docx',
 '~$мментарии.docx',
 'Комментарии (автовосстановление).docx',
 'Комментарии.docx']

In [6]:
ID=filenames[0:2]#вынеси объекты 3:4
ID

['NP102302', 'NP102303']

In [ ]:
#редактируем чтобы были только полные записи 
#02,03,04,05,06,10,11,12,13,14  
del ID [5:7]
print(ID)

In [ ]:
#каналы: 'Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2'
#каналы затылочные O1, O2, Oz, PO3, PO4, P7, P8, P3, P4, Pz
#каналы прецентарльной извилины F3, F4, Fz, C3, C4, Cz 

# Рабочий шаблон с подсчетом PSD и сохранением результатов в таблицу! 

In [37]:
#код для подсечта PSD по Эпохам для условий фракталы с аудио и без по каналам С3 И С4 в частоте 1-15 Гц 
#код с подсчетом psd 
#необходимо изменить только название фафла для обработки и название файлов которые собираешься изменять 
fig, ax = plt.subplots()

#перове условие Название: 
psds_target_list = []
data_variable_first = [] 
data_id = []

for i in range(len(ID)):
    print(ID[i])
    name = f'{ID[i]}/{ID[i]}_all_epochs_new_fractal.fif'
    variable_first = mne.read_epochs(name, proj=True, preload=True, verbose=None)#читаем запись 
    spectrum = variable_first.compute_psd(method='welch', fmin=1, fmax=15, tmin=None, tmax=None, proj=False, n_jobs=1, verbose=None)#считаем PSD
    psds, freqs = spectrum.get_data(return_freqs=True)#Получите все эпохи в виде 3D-массива

    # Find corresponding indices using mne.pick_types() 
    # mne.pick_types определяет индексы каналов ЭЭГ для обращения
    picks_roi_vis = mne.pick_types(variable_first.info, eeg=True, stim=False, exclude='bads', selection=["C3","C4"])
    psds_mean=np.mean(psds, axis=0)#cчитаем сред знач Эпох
    psds_target = psds_mean[picks_roi_vis]#СОЗДАЕМ СПИСОК ИНДЕКСОВ 
    psd_variable_first = 10 * np.log10(psds_target.mean())#считаем PSD для подсчета 
    ID_namber = [f'{ID[i]}_all_epochs_new_fractal.fif']
    data_variable_first.append(psd_variable_first)
    data_id.append(ID_namber)
    print(f'PSD {ID[i]}_all_epochs_new_fractal.fif: {psd_variable_first}')
    
    psds_target_list.append(psds_target[0])#создаем список (списков?) нужных нам отведений 
    ax.plot(freqs, psds_target[0], color='white', alpha=0.1, label=f"PSD {ID[i]}_all_epochs_new_fractal.fif: {psd_variable_first:.2f}")
av_psds_target_list =  np.mean(np.array(psds_target_list), axis=0)#считается усред PSD                             
ax.plot(freqs,  av_psds_target_list, color='b')
avr_PSD_variable_first = 10 * np.log10(av_psds_target_list.mean())#Считается усред PSD для принта 
df = pd.DataFrame({"ID": data_id, "PSD": data_variable_first})
print(f'avr_PSD_variable_first = {avr_PSD_variable_first}')
print(data_variable_first)
print(data_id)
print(df)
df.to_excel('D:/all_eeg/mu_eeg/obrabot/общие графики/draft PSD для фракталов (С3,С4,1-15Гц).xlsx', index=False)

#второе условие Название:   
psds_target_list = []
data_variable_second = []
data_id_2 = []

for i in range(len(ID)):
    print(ID[i])
    name = f'{ID[i]}/{ID[i]}_all_epochs_new_fractal_audio.fif'
    variable_second = mne.read_epochs(name, proj=True, preload=True, verbose=None)
    spectrum = variable_second.compute_psd(method='welch', fmin=1, fmax=15, tmin=None, tmax=None, proj=False, n_jobs=1, verbose=None)
    psds, freqs = spectrum.get_data(return_freqs=True)

    # Find corresponding indices using mne.pick_types() 
    picks_roi_vis = mne.pick_types(variable_second.info, eeg=True, stim=False, exclude='bads', selection=["C3","C4"])
    psds_mean=np.mean(psds, axis=0)   
    psds_target = psds_mean[picks_roi_vis]
    psd_variable_second = 10 * np.log10(psds_target.mean()) 
    ID_namber_2 = [f'{ID[i]}_all_epochs_new_fractal_audio.fif']
    data_variable_second.append(psd_variable_second)
    data_id_2.append(ID_namber_2)
    print(f'PSD {ID[i]}_all_epochs_new_fractal_audio.fif: {psd_variable_second}')
    psds_target_list.append(psds_target[0])
    ax.plot(freqs, psds_target[0], color='white', alpha=0.1, label=f"PSD {ID[i]}_all_epochs_new_fractal_audio.fif: {psd_variable_second:.2f}")
av_psds_target_list =  np.mean(np.array(psds_target_list), axis=0)                              
ax.plot(freqs,  av_psds_target_list, '--b')
avr_PSD_variable_second = 10 * np.log10(av_psds_target_list.mean()) 
df = pd.DataFrame({"ID": data_id_2, "PSD": data_variable_second})
print(f'avr_PSD_variable_second = {avr_PSD_variable_second}')
print(data_variable_second)
print(data_id_2)
print(df)
df.to_excel('D:/all_eeg/mu_eeg/obrabot/общие графики/draft PSD для фракталов с аудио (С3,С4,1-15Гц).xlsx', index=False)


# выводим легенду и оси
ax.legend()
ax.set_xlabel('Freguency (Hz)')
ax.set_ylabel('PSD (fT/cm)^2/Hz (dB)')
ax.set_title(f'PSD для условий фракталы с аудио и без  1-15 по отведениям с3,с4, {len(ID)} человек. avr_PSD_var_first = {avr_PSD_variable_first}, avr_PSD_var_second  = {avr_PSD_variable_second}')
ax.grid(True)

plt.show()


NP102302
Reading D:\all_eeg\mu_eeg\chernovik_2\ssd\NP102302\NP102302_all_epochs_new_fractal.fif ...
    Found the data of interest:
        t =       0.00 ...    1998.00 ms
        0 CTF compensation matrices available
Not setting metadata
29 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.000 (s)
PSD NP102302_all_epochs_new_fractal.fif: -112.13874056119712
NP102303
Reading D:\all_eeg\mu_eeg\chernovik_2\ssd\NP102303\NP102303_all_epochs_new_fractal.fif ...
    Found the data of interest:
        t =       0.00 ...    1998.00 ms
        0 CTF compensation matrices available
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.000 (s)
PSD NP102303_all_epochs_new_fractal.fif: -113.82041553054049
avr_PSD_variable_first = -112.79888642051056
[-112.13874056119712, -113.82041553054049]
[['NP102302_all_epochs_new_fractal.fif'], ['NP102303_all_epochs_new_frac

C:\Users\chuba\AppData\Local\Temp\ipykernel_5828\916221396.py:14: RuntimeWarning: This filename (NP102302/NP102302_all_epochs_new_fractal.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  variable_first = mne.read_epochs(name, proj=True, preload=True, verbose=None)#читаем запись
C:\Users\chuba\AppData\Local\Temp\ipykernel_5828\916221396.py:14: RuntimeWarning: This filename (NP102303/NP102303_all_epochs_new_fractal.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  variable_first = mne.read_epochs(name, proj=True, preload=True, verbose=None)#читаем запись


NP102302
Reading D:\all_eeg\mu_eeg\chernovik_2\ssd\NP102302\NP102302_all_epochs_new_fractal_audio.fif ...
    Found the data of interest:
        t =       0.00 ...    1998.00 ms
        0 CTF compensation matrices available
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.000 (s)
PSD NP102302_all_epochs_new_fractal_audio.fif: -113.12341888907837
NP102303
Reading D:\all_eeg\mu_eeg\chernovik_2\ssd\NP102303\NP102303_all_epochs_new_fractal_audio.fif ...
    Found the data of interest:
        t =       0.00 ...    1998.00 ms
        0 CTF compensation matrices available
Not setting metadata
16 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.000 (s)
PSD NP102303_all_epochs_new_fractal_audio.fif: -113.61042629727265
avr_PSD_variable_second = -113.35187735225263
[-113.12341888907837, -113.61042629727265]
[['NP102302_all_epochs_new_fractal_audio.fif'],

C:\Users\chuba\AppData\Local\Temp\ipykernel_5828\916221396.py:49: RuntimeWarning: This filename (NP102302/NP102302_all_epochs_new_fractal_audio.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  variable_second = mne.read_epochs(name, proj=True, preload=True, verbose=None)
C:\Users\chuba\AppData\Local\Temp\ipykernel_5828\916221396.py:49: RuntimeWarning: This filename (NP102303/NP102303_all_epochs_new_fractal_audio.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  variable_second = mne.read_epochs(name, proj=True, preload=True, verbose=None)
